In [ ]:
from csc4585 import flour

In [ ]:
%%flour
import parallel;

let i = mutex(0);

let inc = share<i> void() {
    let last: int;

    for j in [0..1000] {
        i.lock();
        i.set(i.get() + 1);
        last = i.get();
        i.unlock();
    }

    println("i=" + last);
};

for t in [spawn(inc) | j in [0..100]] {
    t.join();
}

In [ ]:
%%flour
import parallel;

let i = mutex(0);

let inc = share<i> void() {
    let last: int;

    for j in [0..1000] {
        last = acquire<i> int() {
            i.set(i.get() + 1);
            i.get()
        }();
    }

    println("i=" + last);
};

for t in [spawn(inc) | j in [0..100]] {
    t.join();
}

i.lock();
println("AT FINISH: " + i.get());

In [ ]:
%%flour
import parallel;

let i = mutex(0);
let final N = 100;
let final N_INC = 1000;

let inc = acquire<i, N, N_INC> void() {
    for j in [0..N_INC] {
        i.set(i.get() + 1);
    }

    println("i=" + i.get());
    i.notify();
};

let f = spawn(acquire<i, N, N_INC> void() {
    while i.get() != N * N_INC {
        i.wait();
    }
});

for t in [spawn(inc) | j in [0..100]] {
    t.join();
}

f.join();

i.lock();
println("AT FINISH: " + i.get());

In [ ]:
%%flour
import parallel;

let i = mutex(0);
let final N = 100;
let final N_INC = 1000;

let inc = acquire<i, N, N_INC> void() {
    for j in [0..N_INC] {
        i.set(i.get() + 1);
    }

    println("i=" + i.get());
    i.notify();
};

let f = spawn(acquire<i, N, N_INC> void() {
    while i.get() != N * N_INC {
        i.wait();
    }
});

for j in [0..100] {
    spawn(inc);
}

f.join();

i.lock();
println("AT FINISH: " + i.get());